# Auswertung der Daten der Wiener Linien Teil 2

Damit wir die Daten laden können, brauchen wir wieder unsere Funktion *loadFromWeb()*, die die
eingelesene CSV Datei als Dataframe zurückgibt.

In [1]:
import pandas as pd

def loadFromWeb(url):
    return pd.read_csv(url, sep=";", encoding="utf-8")

## Joins und Index

Für die nachfolgenden Beispiele laden wir alle 3 Quellen in 3 DataFrames (stations, steige, linien).
Bei der Ausgabe sehen wir eine klassische Tabellenstruktur, die mit Fremdschlüssel auf andere
Tabellen verweist:

In [2]:
stations = loadFromWeb("https://data.wien.gv.at/csv/wienerlinien-ogd-haltestellen.csv")
steige = loadFromWeb("https://data.wien.gv.at/csv/wienerlinien-ogd-steige.csv")
linien = loadFromWeb("https://data.wien.gv.at/csv/wienerlinien-ogd-linien.csv")
display(stations[0:3])
display(steige[0:3])
display(linien[0:3])

,HALTESTELLEN_ID,TYP,DIVA,NAME,GEMEINDE,GEMEINDE_ID,WGS84_LAT,WGS84_LON,STAND
0,214460106,stop,60200001,Schrankenberggasse,Wien,90001,48.173801,16.389807,NaN
1,214460107,stop,60200002,Achengasse,Wien,90001,48.284526,16.448898,NaN
2,214460108,stop,60200003,Ada-Christen-Gasse,Wien,90001,48.152866,16.385954,NaN


,STEIG_ID,FK_LINIEN_ID,FK_HALTESTELLEN_ID,RICHTUNG,REIHENFOLGE,RBL_NUMMER,BEREICH,STEIG,STEIG_WGS84_LAT,STEIG_WGS84_LON,STAND
0,214689748,214433691,214461074,H,1,4931.0,2.0,U3-H,48.211570,16.311438,NaN
1,214689749,214433691,214461382,H,2,4932.0,1.0,U3-H,48.204584,16.309076,NaN
2,214689750,214433691,214461121,H,3,4933.0,1.0,U3-H,48.199782,16.311366,NaN


,LINIEN_ID,BEZEICHNUNG,REIHENFOLGE,ECHTZEIT,VERKEHRSMITTEL,STAND
0,214433717,D,10,1,ptTram,NaN
1,406201771,N17,0,1,Pt_RufbusNacht,NaN
2,214433953,N20,320,1,ptBusNight,NaN


Nun wollen wir eine Liste der Haltestellen samt Linien, die sie anfahren, erstellen. Dafür müssen wir im klassischen SQL 3 Tabellen verknüpfen:

```sql
SELECT s.Haltestellen_Id, s.Name, s.Gemeinde, s.Gemeinde_Id, l.Bezeichnung, l.Verkehrsmittel
FROM Stations s INNER JOIN Steige st ON (s.HALTESTELLEN_ID = st.FK_HALTESTELLEN_ID)
                INNER JOIN Linien l ON (l.LINIEN_ID = st.FK_LINIEN_ID)
```

Bei Dataframes gibt es auch eine Join Funktion, die solche Operationen erlaubt.
Wir müssen verschiedene Parameter angeben:

- Was ist der zweite Dataframe? Das ist das erste Argument.
- Welche Spalte wollen wir *vom Aufrufer* für den Join verwenden?
- Wie wollen wir verknüpfen (inner, left, right, outer)?
- Welches Suffix soll verwendet werden, wenn gleiche Spaltennamen in beiden Tabellen vorkommen?

Auf https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.join.html ist eine
Beschreibung der *join()* Funktion.

In [3]:
steige.join(stations, on="FK_HALTESTELLEN_ID", how='inner', lsuffix='STEIG')

,STEIG_ID,FK_LINIEN_ID,FK_HALTESTELLEN_ID,RICHTUNG,REIHENFOLGE,RBL_NUMMER,BEREICH,STEIG,STEIG_WGS84_LAT,STEIG_WGS84_LON,STANDSTEIG,HALTESTELLEN_ID,TYP,DIVA,NAME,GEMEINDE,GEMEINDE_ID,WGS84_LAT,WGS84_LON,STAND


Das Ergebnis ist allerdings nicht wie erwartet. Wir bekommen keinen Datensatz. Die Dokumentation
über den parameter *on* gibt Aufschluss:

> Column or index level name(s) in the caller to join on the index in other, otherwise joins index-on-index. If multiple values given, the other DataFrame must have a MultiIndex. Can pass an array as the join key if it is not already contained in the calling DataFrame. Like an Excel VLOOKUP operation.

Es wird also im zweiten Dataframe, der als Argument übergeben wird, der *Index* verwendet. Damit haben wir uns noch nicht genauer beschäftigt.

## Der Index

Als Demonstration legen wir einen Dataframe mit 3 Spalten an: ID, SUBID und VALUE. Dann verwenden
wir die Spalte ID als Index. *set_index()* liefert einen neuen Dataframe zurück, der den Index besitzt.

In [4]:
demoDf = pd.DataFrame({"ID": [0,0,0,1,2], "SUBID": ['a','b','a','a','c'], "VALUE": [10,11,12,13, 14]})
indexedDf = demoDf.set_index("ID")
indexedDf

,SUBID,VALUE
ID,,
0,a,10
0,b,11
0,a,12
1,a,13
2,c,14


Wir sehen also, dass ein Index nicht eindeutig sein muss. Möchten wir nun auf bestimmte Werte von
ID zugreifen, können wir die Spalte *ID* nicht mehr direkt ansprechen. Deswegen gibt es *loc*. Es
erlaubt den Zugriff über den Index. Beachte die eckigen Klammern bei *loc*!

In [5]:
# indexedDf["ID"]    # Key Error: Die Spalte ID existiert nicht mehr, da sie ein Index ist.
indexedDf.loc[0]     # Alle Daten mit dem Indexwert 0 ausgeben.

,SUBID,VALUE
ID,,
0,a,10
0,b,11
0,a,12


Möchten wir die Indizierung wieder rückgängig machen, können wir *reset_index()* verwenden.
Die Option *inplace=True*, ändert den Dataframe direkt, d. h. er wird nicht zurückgegeben.
ID ist nun wieder eine normale Spalte. **Achtung** Bei mehrmaligem Ausführen wird der implizit
angelegte Index (eine Laufnummer) zur Spalte. Verwende daher den *inplace* Parameter nur mit
Bedacht.

Bei *set_index()* haben wir unsere Spalte "verloren". Mit *drop=False* können wir sie behalten,
damit wir weiterhin darauf zugreifen können.

In [6]:
indexedDf.reset_index(inplace=True)
display(indexedDf)
demoDf.set_index("ID", drop=False)

,ID,SUBID,VALUE
0,0,a,10
1,0,b,11
2,0,a,12
3,1,a,13
4,2,c,14


,ID,SUBID,VALUE
ID,,,
0,0,a,10
0,0,b,11
0,0,a,12
1,1,a,13
2,2,c,14


Wir können auch einen *MultiIndex* (auch hierarchical index genannt) erstellen. Er besteht aus 2
Spalten (ID und SUBID):

In [7]:
multiIndexDf = demoDf.set_index(["ID", "SUBID"])
display(multiIndexDf)
display(multiIndexDf.loc[(0, "a")])  # PerformanceWarning: indexing past lexsort depth may impact performance.

VALUE
ID SUBID       
0  a         10
   b         11
   a         12
1  a         13
2  c         14

C:\Users\Michael\AppData\Local\Temp/ipykernel_14928/3969812244.py:3: PerformanceWarning: indexing past lexsort depth may impact performance.
  display(multiIndexDf.loc[(0, "a")])  # PerformanceWarning: indexing past lexsort depth may impact performance.


VALUE
ID SUBID       
0  a         10
   a         12

Diese Warnung ist verständlich, wenn das Konzept des Index aus der Datenbankwelt bereits bekannt
ist. Ist der Dataframe nicht nach dem Index sortiert, sind die Daten quer über den Speicher
verteilt. Die Methode *sort_index()* sortiert nach dem Index und erlaubt einen schnelleren
Zugriff.

Über *loc* können wir mit einem *Tuple* auf beide Indexteile zugreifen. Ein Tupel in Python wird
in Klammer geschrieben und umfasst mehrere Werte, die nicht den selben Datentyp haben müssen.

In [8]:
multiIndexDf.sort_index(inplace=True)
display(multiIndexDf)
display(multiIndexDf.loc[(0, "a")])

VALUE
ID SUBID       
0  a         10
   a         12
   b         11
1  a         13
2  c         14

VALUE
ID SUBID       
0  a         10
   a         12

## Indizierung und Join mit den Haltestellendaten

Diese Anweisung kann nicht mehrmals ausgeführt werden, da wir den originalen Dataframe ändern
und die ID Spalte somit nicht mehr als Spalte vorhanden ist.

In [9]:
stations.set_index("HALTESTELLEN_ID", inplace=True, drop=False)
stations.sort_index(inplace=True)
steige.set_index("STEIG_ID", inplace=True, drop=False)
steige.sort_index(inplace=True)
linien.set_index("LINIEN_ID", inplace=True, drop=False)
linien.sort_index(inplace=True)

Jetzt funktioniert auch unser Join zwischen Steige und Stations. Hinweis: Es wird mit dem Index
des Zielframes (*stations*) verknüpft. Dadurch können wir nicht *stations.join(steige)* schreiben,
da der Index von Steig nicht der Fremdschlüssel ist.

In [10]:
steige.join(stations, on="FK_HALTESTELLEN_ID", how='inner', lsuffix='STEIG')

,STEIG_ID,FK_LINIEN_ID,FK_HALTESTELLEN_ID,RICHTUNG,REIHENFOLGE,RBL_NUMMER,BEREICH,STEIG,STEIG_WGS84_LAT,STEIG_WGS84_LON,STANDSTEIG,HALTESTELLEN_ID,TYP,DIVA,NAME,GEMEINDE,GEMEINDE_ID,WGS84_LAT,WGS84_LON,STAND
STEIG_ID,,,,,,,,,,,,,,,,,,,,
214689748,214689748,214433691,214461074,H,1,4931.0,2.0,U3-H,48.211570,16.311438,NaN,214461074,stop,60200981,Ottakring,Wien,90001,48.212025,16.311672,NaN
214689814,214689814,214433691,214461074,R,21,4930.0,2.0,U3-R,48.211552,16.311591,NaN,214461074,stop,60200981,Ottakring,Wien,90001,48.212025,16.311672,NaN
214692215,214692215,214433765,214461074,H,10,1384.0,3.0,46-H,48.212588,16.312525,NaN,214461074,stop,60200981,Ottakring,Wien,90001,48.212025,16.311672,NaN
214692240,214692240,214433765,214461074,R,4,1387.0,3.0,46-R,48.212606,16.311807,NaN,214461074,stop,60200981,Ottakring,Wien,90001,48.212025,16.311672,NaN
219364907,219364907,219364886,214461074,H,1,8898.0,3.0,45B,48.211875,16.310872,NaN,214461074,stop,60200981,Ottakring,Wien,90001,48.212025,16.311672,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532804430,532804430,214433747,214460131,R,2,142.0,0.0,40-R,48.231728,16.324176,NaN,214460131,stop,60200026,Alsegger Straße,Wien,90001,48.231686,16.324365,NaN
532804408,532804408,214433747,214460607,H,14,NaN,0.0,40-H,48.232470,16.318014,NaN,214460607,stop,60200502,Herbeckstraße,Wien,90001,48.232464,16.317502,NaN
532804410,532804410,214433747,214460607,H,16,NaN,0.0,40-R,48.232452,16.317026,NaN,214460607,stop,60200502,Herbeckstraße,Wien,90001,48.232464,16.317502,NaN


Wir können auch mehrmals die *join()* Funktion verwenden. Wir gehen vom Dataframe aus, der den
Fremdschlüssel beinhaltet (also *steige*). Dann verknüpfen wir zu den entsprechenden Dataframes,
die diesen Schlüssel als Primärschlüssel (Index) verwenden.

Im Zielframe brauchen wir den Index mit der Steig ID nicht mehr, daher löschen wir ihn mit
*reset_index()*. Da wir die Steig ID als Spalte behalten haben, geben wir *drop=True* an. Sonst
würde ein Fehler entstehen, da die Spalte STEIG_ID schon vorhanden ist.

Mit *rename()* können wir ein Dictionary angeben, welches Spalten umbenennt. Das macht die Arbeit
leichter, da wir bessere Bezeichnungen zuweisen können.

Durch den Join entstehen mehrere Einträge für die selbe Linie pro Station (der Steig ist für
Hin- und Rückrichtung angelegt, deswegen verdoppeln sich die Einträge). Mit *drop_duplicates()*
können wir eine Spaltenliste angeben, die im Zielframe nur einmal vorkommen darf.

In [11]:
# Schritt 1: Join mit indizierten Dataframes
stationsMitLinien = steige.join(stations, on="FK_HALTESTELLEN_ID", how='inner', rsuffix='STATION') \
    .join(linien, on="FK_LINIEN_ID", how='inner', rsuffix='LINIE')
# Schritt 2: Spalten auswählen, da wir nicht alle brauchen. Einige Spalten werden umbenannt
stationsMitLinien = stationsMitLinien[["GEMEINDE", "GEMEINDE_ID", "NAME", "HALTESTELLEN_ID", "LINIEN_ID", "BEZEICHNUNG", "VERKEHRSMITTEL"]] \
    .rename(columns={"BEZEICHNUNG": "LINIE", "VERKEHRSMITTEL": "LINIENART"})
# Schritt 3: Den Index der Steigtabelle löschen (unnötig) und Duplikate entfernen.
stationsMitLinien = stationsMitLinien.drop_duplicates(["HALTESTELLEN_ID", "LINIEN_ID"]).reset_index(drop=True)
stationsMitLinien[0:5]

,GEMEINDE,GEMEINDE_ID,NAME,HALTESTELLEN_ID,LINIEN_ID,LINIE,LINIENART
0,Wien,90001,Ottakring,214461074,214433691,U3,ptMetro
1,Wien,90001,Kendlerstraße,214461382,214433691,U3,ptMetro
2,Wien,90001,Hütteldorfer Straße,214461121,214433691,U3,ptMetro
3,Wien,90001,Johnstraße,214460711,214433691,U3,ptMetro
4,Wien,90001,Schweglerstraße,214461278,214433691,U3,ptMetro


Nun aggregieren wir die Daten, indem wir die Anzahl der Datensätze pro Linienart zählen. Wie in SQL
müssen wir alle Spalten angeben, die wir in der Ausgabe noch haben wollen.

In [12]:
linienarten = stationsMitLinien.groupby(["GEMEINDE", "GEMEINDE_ID", "NAME", "HALTESTELLEN_ID", "LINIENART"], as_index=False) \
    .aggregate("size") \
    .rename(columns = {"size": "LINIENART_COUNT"})
linienarten[linienarten.NAME == "Oper/Karlsplatz U"]

,GEMEINDE,GEMEINDE_ID,NAME,HALTESTELLEN_ID,LINIENART,LINIENART_COUNT
1907,Wien,90001,Oper/Karlsplatz U,214461068,ptBadner_Bahn,1
1908,Wien,90001,Oper/Karlsplatz U,214461068,ptBusCity,2
1909,Wien,90001,Oper/Karlsplatz U,214461068,ptBusNight,9
1910,Wien,90001,Oper/Karlsplatz U,214461068,ptTram,6


## Pivotierung

Bei einer begrenzten Anzahl an Wertausprägungen (die Linienart hat ptTram, ptBusCity, ...) ist es
oft einfacher, diese Werte als Spalten anzulegen. Der Klassiker für eine Pivotierung ist eine Spalte
m und w für männlich und weiblich. Auch hier gibt es nur 2 Wertausprägungen. Mit der Funktion
*pivot()* geben wir zuerst als Index die Spalten an, die wir weiter haben wollen. Die neuen
Spalten sollen die Wertausprägungen von *LINIENART* sein, deswegen geben wir diese Spalte bei
*columns* an. Der Wert der Spalte soll von *LINIENART_COUNT* genommen werden.

Das Ergebnis hat natürlich viele *NaN* Werte, da nicht jede Linienart bei jeder Haltestelle vorhanden
ist. Mit *fillna()* ersetzen wir diese Werte durch 0. Danach konvertieren wir die Werte in ein
int zurück. Damit *NaN* gespeichert werden kann, musste der Datentyp geändert werden.

Mit *reset_index()* erzeugen wir wieder normale Spalten aus dem erzeugten Index. Dann wird auch
der Name der Achse zurückgesetzt (wäre sonst *LINIENART*).

Tipp: Sieh dir die Ausgabe nach jedem Schritt an, um die Anweisung besser zu verstehen.

In [13]:
pivoted = linienarten.pivot(index=["GEMEINDE", "GEMEINDE_ID", "NAME", "HALTESTELLEN_ID"], columns="LINIENART", values="LINIENART_COUNT") \
    .fillna(0) \
    .astype(int) \
    .reset_index().rename_axis(None, axis=1)

pivoted.columns = pivoted.columns.str.upper()  # Die Spaltennamen wollen wir alle wieder groß schreiben.
pivoted[pivoted.PTBADNER_BAHN > 0][0:5]        # Wo gibt es Stationen mit der Linie vom Typ PTBADNER_BAHN

,GEMEINDE,GEMEINDE_ID,NAME,HALTESTELLEN_ID,PT_RUFBUSNACHT,PTBADNER_BAHN,PTBUSCITY,PTBUSNIGHT,PTMETRO,PTTRAINS,PTTRAM,PTTRAMVRT,PT_RUFBUSTAG
6,Baden bei Wien,30604,Baden Josefsplatz,214462070,0,1,0,0,0,0,0,0,0
7,Baden bei Wien,30604,Baden Landesklinikum,219364399,0,1,0,0,0,0,0,0,0
8,Baden bei Wien,30604,Baden Leesdorf,214462091,0,1,0,0,0,0,0,0,0
9,Baden bei Wien,30604,Baden Melkergründe,214464749,0,1,0,0,0,0,0,0,0
10,Baden bei Wien,30604,Baden Viadukt,214462083,0,1,0,0,0,0,0,0,0


Jetzt können wir besonders einfach weitere Auswertungen machen. Gibt es Stationen, in denen
Linien vom Typ PTBADNER_BAHN und PTBUSNIGHT (Nachtbus) halten?

In [14]:
pivoted[(pivoted.PTBADNER_BAHN > 0)&(pivoted.PTBUSNIGHT > 0)]

,GEMEINDE,GEMEINDE_ID,NAME,HALTESTELLEN_ID,PT_RUFBUSNACHT,PTBADNER_BAHN,PTBUSCITY,PTBUSNIGHT,PTMETRO,PTTRAINS,PTTRAM,PTTRAMVRT,PT_RUFBUSTAG
290,Wien,90001,Aßmayergasse,214460208,0,1,1,1,0,0,1,0,0
464,Wien,90001,Dörfelstraße,214460967,0,1,2,1,0,0,1,0,0
482,Wien,90001,Eichenstraße,214460372,0,1,1,1,0,0,3,0,0
551,Wien,90001,Flurschützstr./Längenfeldgasse,214460473,0,1,1,1,0,0,1,0,0
669,Wien,90001,Gutheil-Schoder-Gasse,214460549,0,1,2,1,0,0,0,0,0
847,Wien,90001,Johann-Strauß-Gasse,214460709,0,1,1,1,0,0,2,0,0
910,Wien,90001,Karlsplatz,214460762,0,1,2,4,2,0,2,0,0
1017,Wien,90001,Laurenzgasse,214460859,0,1,0,1,0,0,2,0,0
1128,Wien,90001,Matzleinsdorfer Platz,214460952,0,1,1,2,0,5,4,0,0
1137,Wien,90001,Mayerhofgasse,214460961,0,1,0,1,0,0,2,0,0


Wo halten die meisten U Bahn Linien?

In [15]:
pivoted[pivoted.PTMETRO == pivoted.PTMETRO.max()]

,GEMEINDE,GEMEINDE_ID,NAME,HALTESTELLEN_ID,PT_RUFBUSNACHT,PTBADNER_BAHN,PTBUSCITY,PTBUSNIGHT,PTMETRO,PTTRAINS,PTTRAM,PTTRAMVRT,PT_RUFBUSTAG
910,Wien,90001,Karlsplatz,214460762,0,1,2,4,2,0,2,0,0
1090,Wien,90001,Längenfeldgasse,214460923,0,0,1,2,2,0,0,0,0
1333,Wien,90001,Praterstern,214461125,1,0,3,2,2,5,2,0,0
1484,Wien,90001,Schottenring,214461261,0,0,1,5,2,0,3,0,0
1500,Wien,90001,Schwedenplatz,214461276,0,0,1,6,2,0,2,1,0
1596,Wien,90001,Spittelau,214461145,1,0,2,2,2,1,1,0,0
1630,Wien,90001,Stephansplatz,214461385,0,0,3,0,2,0,0,0,0
1786,Wien,90001,Westbahnhof,214461519,1,0,0,3,2,1,6,0,0
1796,Wien,90001,Wien Mitte-Landstraße,214460847,0,0,1,1,2,5,1,0,0


## Andere Merge Methoden

In Pandas gibt es nicht nur den Join, um Dataframes zu verknüpfen. Mit *merge()* gibt es eine
sehr mächtige Methode, die sehr gut dokumentiert ist:
https://pandas.pydata.org/docs/user_guide/merging.html#database-style-dataframe-or-named-series-joining-merging

Für Zeitreihen steht auch *merge_asof()* zur Verfügung, die die nächstgelegenen Werte herausfinden
kann:
https://pandas.pydata.org/docs/user_guide/merging.html#merging-asof